In [1]:
#noise removal process code
import os
import cv2
import numpy as np

def kapur_entropy_thresholding(image):
    hist = cv2.calcHist([image], [0], None, [256], [0, 256]).flatten()
    total_pixels = np.sum(hist)
    probabilities = hist / total_pixels
    max_entropy = -np.inf
    optimal_threshold = 0

    for t in range(1, 256):
        P1 = np.sum(probabilities[:t])
        if P1 == 0:
            continue
        H1 = -np.sum((probabilities[:t] / P1) * np.log(probabilities[:t] / P1 + 1e-10))

        P2 = np.sum(probabilities[t:])
        if P2 == 0:
            continue
        H2 = -np.sum((probabilities[t:] / P2) * np.log(probabilities[t:] / P2 + 1e-10))

        total_entropy = H1 + H2

        if total_entropy > max_entropy:
            max_entropy = total_entropy
            optimal_threshold = t

    return optimal_threshold

def create_lesion_mask(image, adaptive_threshold):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_gray = clahe.apply(gray)
    blurred = cv2.medianBlur(enhanced_gray, 5)

    sobelx = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobelx ** 2 + sobely ** 2)

    # Calculate the average of gradient_magnitude as the threshold
    sobel_threshold = np.mean(gradient_magnitude)

    # Create edge mask based on the average threshold
    edge_mask = np.zeros_like(gray, dtype=np.uint8)
    edge_mask[gradient_magnitude > sobel_threshold] = 255

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    a_channel = lab[:, :, 1]
    mean_a = np.mean(a_channel)
    is_red_dominant = mean_a > 128

    mask = np.zeros_like(blurred)
    if is_red_dominant:
        mask[a_channel > mean_a] = 255
    else:
        mask[blurred <= adaptive_threshold] = 255

    mask = cv2.bitwise_or(mask, edge_mask)

    kernel_size = max(5, image.shape[0] // 100)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_area = image.shape[0] * image.shape[1] * 0.01
    for contour in contours:
        if cv2.contourArea(contour) < min_area:
            cv2.drawContours(mask, [contour], -1, 0, -1)

    return mask


def segment_lesion(image, mask):
    segmented = np.zeros_like(image)
    segmented[mask == 255] = image[mask == 255]
    return segmented

# Updated noise removal function for efficient and accurate noise pixel replacement
def remove_noise(image, mask):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply black top-hat transform
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 3))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    # Threshold the black top-hat result to create a noise mask
    _, noise_mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    
    # Restrict noise to lesion region
    noise_mask = cv2.bitwise_and(noise_mask, mask)

    # Calculate the percentage of noisy pixels
    noise_percentage = (np.sum(noise_mask) / 255) / np.sum(mask / 255) * 100

    # If noise percentage is below a threshold, return the original image
    if noise_percentage < 1.0:  # You can adjust this threshold as needed
        return image

    # If noise is detected, proceed with noise removal as before
    result = image.copy()
    filtered_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    noise_coords = np.column_stack(np.where(noise_mask == 255))
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

    for y, x in noise_coords:
        valid_neighbors = []
        search_radius = 1
        
        while not valid_neighbors:
            for dy in range(-search_radius, search_radius + 1):
                for dx in range(-search_radius, search_radius + 1):
                    ny, nx = y + dy, x + dx
                    if 0 <= ny < image.shape[0] and 0 <= nx < image.shape[1] and mask[ny, nx] == 255 and noise_mask[ny, nx] == 0:
                        valid_neighbors.append(filtered_image[ny, nx])
            
            if not valid_neighbors:
                search_radius += 1
        
        result[y, x] = np.mean(valid_neighbors, axis=0).astype(np.uint8)

    return result



def process_image(image_path, output_folder, class_name):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (150, 150))

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist_normalized = cv2.calcHist([gray_image], [0], None, [256], [0, 256]).flatten() / gray_image.size

    max_sigma, otsu_threshold = 0, 0
    for T in range(1, 256):
        P1, P2 = hist_normalized[:T].sum(), hist_normalized[T:].sum()
        if P1 == 0 or P2 == 0:
            continue
        mu1 = np.dot(np.arange(T), hist_normalized[:T]) / P1
        mu2 = np.dot(np.arange(T, 256), hist_normalized[T:]) / P2
        sigma_b_squared = P1 * P2 * (mu1 - mu2) ** 2
        if sigma_b_squared > max_sigma:
            max_sigma, otsu_threshold = sigma_b_squared, T

    kapur_threshold = kapur_entropy_thresholding(gray_image)
    image_entropy = -np.sum(hist_normalized * np.log(hist_normalized + 1e-10))
    w_kapur = image_entropy / (image_entropy + max_sigma)
    w_otsu = max_sigma / (image_entropy + max_sigma)
    adaptive_threshold = int(w_kapur * kapur_threshold + w_otsu * otsu_threshold)

    mask = create_lesion_mask(image, adaptive_threshold)
    segmented = segment_lesion(image, mask)
    noise_removed = remove_noise(segmented, mask)

    class_output_folder = os.path.join(output_folder, class_name)
    os.makedirs(class_output_folder, exist_ok=True)

    noise_removed_path = os.path.join(class_output_folder, f"noise_removed_{os.path.basename(image_path)}")
    cv2.imwrite(noise_removed_path, noise_removed)

def process_all_images(input_folder):
    output_folder = os.path.join(os.path.dirname(input_folder), "Optimized_output_final")
    os.makedirs(output_folder, exist_ok=True)
    
    class_counts = {}
    total_images = 0

    for class_name in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_name)
        if os.path.isdir(class_path):
            image_count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_name] = image_count
            total_images += image_count

    print("\nImage count per class:")
    print("-" * 30)
    print(f"{'Class Name':<20}{'Count':>10}")
    print("-" * 30)
    for class_name, count in class_counts.items():
        print(f"{class_name:<20}{count:>10}")
    print("-" * 30)
    print(f"{'Total images:':<20}{total_images:>10}")


    for class_name in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_name)
        if os.path.isdir(class_path):
            print(f"Processing images in {class_name}")
            for image_name in os.listdir(class_path):
                if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(class_path, image_name)
                    process_image(image_path, output_folder, class_name)

    print("\nProcessing complete. Noise removed images saved in 'Optimized_output_final' folder.")

if __name__ == "__main__":
    input_folder = "F:\IMG_CLASSES"
    process_all_images(input_folder)





Image count per class:
------------------------------
Class Name               Count
------------------------------
1. Eczema 1677            1677
10. Warts Molluscum and other Viral Infections - 2103      2103
2. Melanoma 15.75k        3140
3. Atopic Dermatitis - 1.25k      1257
4. Basal Cell Carcinoma (BCC) 3323      3323
5. Melanocytic Nevi (NV) - 7970      7970
6. Benign Keratosis-like Lesions (BKL) 2624      2079
7. Psoriasis pictures Lichen Planus and related diseases - 2k      2055
8. Seborrheic Keratoses and other Benign Tumors - 1.8k      1847
9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k      1702
------------------------------
Total images:            27153
Processing images in 1. Eczema 1677
Processing images in 10. Warts Molluscum and other Viral Infections - 2103
Processing images in 2. Melanoma 15.75k
Processing images in 3. Atopic Dermatitis - 1.25k
Processing images in 4. Basal Cell Carcinoma (BCC) 3323
Processing images in 5. Melanocytic Nevi (NV) 